In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D

In [2]:
height, width = 5, 5
X = np.random.randint(10, size=(height, width))
print(X.shape)
print(X)

(5, 5)
[[0 6 8 0 9]
 [5 6 2 2 7]
 [1 1 3 0 7]
 [3 5 1 1 5]
 [6 0 7 3 8]]


In [3]:
window_height, window_width = 3, 3

for i in range(height - window_height + 1):
    for j in range(width - window_width + 1):
        current_window = X[i : i + window_height, j : j + window_width]
        print(f"W : {i}{i + window_height},{j}{j + window_width}, \n {current_window} \n")

W : 03,03, 
 [[0 6 8]
 [5 6 2]
 [1 1 3]] 

W : 03,14, 
 [[6 8 0]
 [6 2 2]
 [1 3 0]] 

W : 03,25, 
 [[8 0 9]
 [2 2 7]
 [3 0 7]] 

W : 14,03, 
 [[5 6 2]
 [1 1 3]
 [3 5 1]] 

W : 14,14, 
 [[6 2 2]
 [1 3 0]
 [5 1 1]] 

W : 14,25, 
 [[2 2 7]
 [3 0 7]
 [1 1 5]] 

W : 25,03, 
 [[1 1 3]
 [3 5 1]
 [6 0 7]] 

W : 25,14, 
 [[1 3 0]
 [5 1 1]
 [0 7 3]] 

W : 25,25, 
 [[3 0 7]
 [1 1 5]
 [7 3 8]] 



<img src="https://drive.google.com/uc?export=download&id=1pLARvvjQopgge05NFzi2t-eRQhPFvbwZ">
<img src="https://drive.google.com/uc?export=download&id=1Sqh6yrWp_XJ5MtH-GsDwmP5VdKChIplK">

In [4]:
batch_size, img_height, img_width, img_channel = 1, 5, 5, 1
kernel_size = 3
filters = 1

X = tf.random.uniform(shape=(batch_size, img_height, img_width, img_channel),
                      minval=0, maxval=10, dtype=tf.float32)
print(X.shape) ## 5 by 5 gray scale image

conv_layer = Conv2D(filters=filters, kernel_size=kernel_size) ## filter의 수 : 뉴런의수, kernel_size : weight matrix의 shape
output = conv_layer(X)

print(output.shape) ## img_size - kernel_size + 1

(1, 5, 5, 1)
(1, 3, 3, 1)


<img src="https://drive.google.com/uc?export=download&id=1sWbSmhtTtxDypIyvV8s6ATrIeycqj2p9">

In [6]:
W, B = conv_layer.get_weights()
print(W.shape, B.shape)

input_np = X.numpy().squeeze()
W_np = W.squeeze()

output_np = np.zeros(shape=(img_height - kernel_size + 1, img_width - kernel_size + 1))

for i in range(img_height - kernel_size + 1): ## 5 - 3 + 1 = 4
    for j in range(img_width - kernel_size + 1):
        current_window = input_np[i : i + kernel_size, j : j + kernel_size] ## slicing은 end - 1 까지 가져온다!!
        output_np[i, j] = np.sum(current_window * W_np) + B ## 현재 window * kernel + b = single value ---> output matirx's element.

print(output_np.shape)

(3, 3)


In [8]:
## Multiple kernel
## 1개의 kernel 당 1개의 output matrix가 생성된다.

batch_size, img_height, img_width, img_channel = 1, 5, 5, 1
kernel_size = 3
filters = 7

X = tf.random.uniform(shape=(batch_size, img_height, img_width, img_channel),
                      minval=0, maxval=10, dtype=tf.float32)
print(X.shape) ## 5 by 5 gray scale image

conv_layer = Conv2D(filters=filters, kernel_size=kernel_size) ## filter의 수 : 뉴런의 수, kernel_size : weight matrix의 shape
output = conv_layer(X)

print(output.shape) ## (img_size - kernel_size + 1), 1 * 3 * 3 * 7

(1, 5, 5, 1)
(1, 3, 3, 7)


In [14]:
W, B = conv_layer.get_weights()
print(W.shape, B.shape) ## 1 * 3 * 3 * 7, 7

input_np = X.numpy().squeeze()
W_np = W.squeeze()

output_np = np.zeros(shape=(img_height - kernel_size + 1, img_width - kernel_size + 1, filters))

for filter in range(filters):
    current_filter_weight = W_np[ :, :, filter]
    current_filter_bias = B[filter]

    for i in range(img_height - kernel_size + 1): ## 5 - 3 + 1 = 4
        for j in range(img_width - kernel_size + 1):
            current_window = input_np[i : i + kernel_size, j : j + kernel_size] ## slicing은 end - 1 까지 가져온다!!
            
            ## 현재 window * kernel + b = single value ---> output matirx's element.
            output_np[i, j, filter] = np.sum(current_window * current_filter_weight) + current_filter_bias

print(output_np.shape)

(3, 3, 1, 7) (7,)
(3, 3, 7)


In [15]:
## single RGB image & single kernel

batch_size, img_height, img_width, img_channel = 1, 5, 5, 3
kerenl_size = 3
filters = 1

X = tf.random.uniform(shape=(batch_size, img_height, img_width, img_channel),
                      minval=0, maxval=10, dtype=tf.float32)
print(X.shape)

conv_layer = Conv2D(filters=filters, kernel_size=kernel_size)
output = conv_layer(X)
print(output.shape) ## 1 3 3 1

(1, 5, 5, 3)
(1, 3, 3, 1)


In [20]:
W, B = conv_layer.get_weights()
print(W.shape, B.shape) ## 3 3 3 1, 1
W_np = W.squeeze()

X_np = X.numpy().squeeze()
output_np = np.zeros(shape=(img_height - kernel_size + 1, img_width - kernel_size + 1))

for i in range(img_height - kerenl_size + 1):
    for j in range(img_width - kernel_size + 1):
        window = X_np[i : i + kernel_size, j : j + kernel_size, :]
        output_np[i, j] = np.sum(window * W_np) + B

print(output_np.shape)

(3, 3, 3, 1) (1,)
(3, 3)


In [21]:
## Multiple kernel & single RGB image

batch_size, img_height, img_width, img_channel = 1, 5, 5, 3
kerenl_size = 3
filters = 10

X = tf.random.uniform(shape=(batch_size, img_height, img_width, img_channel),
                      minval=0, maxval=10, dtype=tf.float32)
print(X.shape)

conv_layer = Conv2D(filters=filters, kernel_size=kernel_size)
output = conv_layer(X)
print(output.shape) ## 1 3 3 10

(1, 5, 5, 3)
(1, 3, 3, 10)


In [24]:
W, B = conv_layer.get_weights()
print(W.shape, B.shape) ## 3 3 3 10, 10
W_np = W.squeeze()

X_np = X.numpy().squeeze()
output_np = np.zeros(shape=(img_height - kernel_size + 1, img_width - kernel_size + 1, filters))

for filter in range(filters):
    tmp_w, tmp_b = W_np[:, :, :, filter], B[filter] ## height, width, channel, neurons

    for i in range(img_height - kerenl_size + 1):
        for j in range(img_width - kernel_size + 1):
            window = X_np[i : i + kernel_size, j : j + kernel_size, :]
            output_np[i, j, filter] = np.sum(window * tmp_w) + tmp_b

print(output_np.shape)

(3, 3, 3, 10) (10,)
(3, 3, 10)


In [25]:
## Multiple kernel & batch-size RGB image

batch_size, img_height, img_width, img_channel = 10, 5, 5, 3
kerenl_size = 3
filters = 10

X = tf.random.uniform(shape=(batch_size, img_height, img_width, img_channel),
                      minval=0, maxval=10, dtype=tf.float32)
print(X.shape)

conv_layer = Conv2D(filters=filters, kernel_size=kernel_size) ## 3, 3, 3, 10
output = conv_layer(X)

W, B = conv_layer.get_weights()
print(W.shape, B.shape)
print(output.shape) ## 10 3 3 10

(10, 5, 5, 3)
(3, 3, 3, 10) (10,)
(10, 3, 3, 10)
